<a href="https://colab.research.google.com/github/Wency-WangC/Nice-Little-Tools/blob/main/check_booklist_and_catalogue_v1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML
import io, base64, time, urllib.parse

# —— 全局变量 ——
booklist_df = pd.DataFrame()
booklist_df_processed = pd.DataFrame()
booklist_titles = []
booklist_isbns = []
request_dict = {}
current_idx = 0
found_list = []
request_isbns = set()
req_list_only = []
not_found_list = []
header_list = []  # 存原始书单的列标题

# —— 输出区 ——
out_preview    = widgets.Output()
out_col_select = widgets.Output()
out_request    = widgets.Output()
output_area    = widgets.Output()

# —— 1. 上传书单 Excel ——
book_upload = widgets.FileUpload(
    accept='.xlsx', multiple=False, description='选择书单Excel'
)

name_col_input  = widgets.Text(placeholder='书名列 (如 A)')
isbn_col_input  = widgets.Text(placeholder='ISBN列 (如 B)')
col_confirm_btn = widgets.Button(description="确认列")

request_choice     = widgets.RadioButtons(options=['否','是'], value='否')
request_confirm_btn= widgets.Button(description="确认")
request_upload     = widgets.FileUpload(accept='.xlsx', multiple=False, description='选择RequestList')

# 回调：上传书单后读取并预览带字母前缀的表头 & 前5行

def on_book_upload_change(change):
    global booklist_df, header_list
    with out_preview:
        out_preview.clear_output()
        if not book_upload.value:
            return
        fi = list(book_upload.value.values())[0] if isinstance(book_upload.value, dict) else book_upload.value[0]
        if not fi:
            print("❌ 无效文件内容")
            return
        content = fi['content']
        try:
            df = pd.read_excel(io.BytesIO(content), header=0)
        except Exception as e:
            print("❌ 读取失败，请确认Excel：", e)
            return
        booklist_df = df.copy()
        header_list = df.columns.tolist()
        print(f"✅ 书单上传，{len(df)}行，{len(df.columns)}列")
        preview = df.head(5).copy()
        preview.columns = [f"{chr(ord('A')+i)}. {header_list[i]}" for i in range(len(header_list))]
        display(HTML("<b>前5行数据预览（列名带前缀 A., B., C., ...）：</b>"))
        display(preview)
    with out_col_select:
        out_col_select.clear_output()
        display(HTML("<b>请输入书名列和ISBN列 (如 A 和 B)：</b>"))
        display(widgets.HBox([name_col_input, isbn_col_input, col_confirm_btn]))
        name_col_input.on_submit(lambda t: col_confirm_btn.click())
        isbn_col_input.on_submit(lambda t: col_confirm_btn.click())

book_upload.observe(on_book_upload_change, names='value')

# 回调：确认列号后提取数据，并询问Request List上传

def on_col_confirm(b):
    a = name_col_input.value.strip().upper()
    b = isbn_col_input.value.strip().upper()
    if not a or not b:
        print("❌ 请输入完整的列号")
        return
    idx_name = ord(a)-ord('A')
    idx_isbn = ord(b)-ord('A')
    print(f"✅ 列选择：书名列{a}->{idx_name}, ISBN列{b}->{idx_isbn}")
    global booklist_df_processed, booklist_titles, booklist_isbns
    first_title = str(booklist_df.iloc[0, idx_name]).strip().lower()
    first_isbn  = str(booklist_df.iloc[0, idx_isbn]).strip().lower()
    if first_title in ['title','书名','题名'] or first_isbn in ['isbn','isbn13']:
        booklist_df_processed = booklist_df.drop(0).reset_index(drop=True)
        print("▶ 跳过首行表头")
    else:
        booklist_df_processed = booklist_df.copy()
    booklist_df_processed.columns = header_list
    booklist_titles = booklist_df_processed.iloc[:, idx_name].astype(str).tolist()
    booklist_isbns  = booklist_df_processed.iloc[:, idx_isbn].astype(str).tolist()
    print(f"▶ 共 {len(booklist_titles)} 本书待查询")
    display(HTML("<b>是否上传 Request List？</b>"))
    display(request_choice, request_confirm_btn)

col_confirm_btn.on_click(on_col_confirm)

# 回调：Request List选择后

def on_request_confirm(b):
    if request_choice.value=='是':
        display(request_upload, out_request)
    else:
        global request_dict
        request_dict = set()
        start_processing()

request_confirm_btn.on_click(on_request_confirm)

# 回调：上传Request List后收集全表值

def on_request_upload(change):
    with out_request:
        out_request.clear_output()
        if not request_upload.value:
            return
        fi = list(request_upload.value.values())[0] if isinstance(request_upload.value, dict) else request_upload.value[0]
        content = fi['content']
        try:
            df_req = pd.read_excel(io.BytesIO(content), header=None)
        except Exception as e:
            print("❌ Request List读取失败：", e)
            return
        print(f"✅ Request List: {df_req.shape[0]}行 x {df_req.shape[1]}列")
        global request_isbns
        request_isbns.clear()
        for cell in df_req.values.flatten():
            s = str(cell).strip()
            if s and s.lower()!='nan':
                request_isbns.add(s)
        print(f"🔍 收集 {len(request_isbns)} 条Request List值")
        start_processing()

request_upload.observe(on_request_upload, names='value')

# 辅助：生成Catalogue搜索链接
def generate_search_url(q):
    base = "https://librarysearch.library.utoronto.ca/discovery/search"
    params = {"query":f"any,contains,{q}","vid":"01UTORONTO_INST:UTORONTO","search_scope":"MyInstitution","tab":"LibraryCatalog","offset":"0"}
    return f"{base}?"+"&".join(f"{k}={urllib.parse.quote(v)}" for k,v in params.items())

# 主流程

def start_processing():
    display(output_area)
    process_book()

def process_book():
    global current_idx
    with output_area:
        output_area.clear_output()
        if current_idx>=len(booklist_titles):
            finalize_results()
            return
        title = booklist_titles[current_idx]
        isbn  = booklist_isbns[current_idx]
        url_t = generate_search_url(title)
        url_i = generate_search_url(isbn)
        display(HTML(f"<b>第 {current_idx+1}/{len(booklist_titles)} 本：</b>{title} | ISBN: {isbn}<br>"
                     f"🔎 <a href='{url_t}' target='_blank'>按书名检索</a> &nbsp; "
                     f"<a href='{url_i}' target='_blank'>按ISBN检索</a><br><br>""找到这本书了吗？"))
        yes = widgets.Button(description="Yes")
        no  = widgets.Button(description="No")
        display(widgets.HBox([yes,no]))
        def on_yes(b):
            yes.disabled=no.disabled=True
            found_list.append((current_idx, url_i))
            display(HTML(f"<p style='color:green;'>第 {current_idx+1} 行 <b>{title}</b> → Catalogue命中</p>"))
            current_next()
        def on_no(b):
            yes.disabled=no.disabled=True
            if isbn in request_isbns:
                req_list_only.append(current_idx)
                display(HTML(f"<p style='color:green;'>第 {current_idx+1} 行 <b>{title}</b> → Request List命中</p>"))
            else:
                not_found_list.append(current_idx)
                display(HTML(f"<p style='color:red;'>第 {current_idx+1} 行 <b>{title}</b> → 均未命中</p>"))
            time.sleep(1)
            current_next()
        yes.on_click(on_yes)
        no.on_click(on_no)

def current_next():
    global current_idx
    current_idx += 1
    process_book()

# 5. 导出

def finalize_results():
    df_orig = booklist_df_processed.copy().reset_index(drop=True)
    cat_idxs = [idx for idx,_ in found_list]
    cat_urls = [url for _,url in found_list]
    df_cat = df_orig.iloc[cat_idxs].copy().reset_index(drop=True)
    df_cat['链接'] = cat_urls
    df_req = df_orig.iloc[req_list_only].copy().reset_index(drop=True)
    df_not = df_orig.iloc[not_found_list].copy().reset_index(drop=True)
    buf = io.BytesIO()
    with pd.ExcelWriter(buf, engine='openpyxl') as writer:
        df_cat.to_excel(writer, sheet_name="Catalogue有", index=False)
        df_req.to_excel(writer, sheet_name="RequestList有", index=False)
        df_not.to_excel(writer, sheet_name="均未找到", index=False)
    buf.seek(0)
    b64 = base64.b64encode(buf.read()).decode()
    link = f'<a download="result_full.xlsx" style="font-size:18px;" href="data:application/octet-stream;base64,{b64}">📥 下载含原始所有列的结果Excel</a>'
    display(HTML(link))
    print("✅ 全部完成，Excel 已生成")

# 渲染 UI
display(HTML("<h4>1️⃣ 上传书单</h4>"))
display(book_upload)
display(out_preview)
display(out_col_select)
